### Optimization-MLE-Mass_and_Width code
#### Author: Oscar Altuve (ULA - VENEZUELA 2021)
DISCLAIMER:
Under no circumstances does it qualify to reproduce actual ATLAS analysis results or produce publishable results!

Antes de ejecutar el cuaderno se debe correr el código de "Analysis" y "Plotting". Debe asegurarse de que el código de “Analysis” en la celda 6, esté configurado a la creación del histograma “mass_four_lep” entre el rango (110, 135) GeV y 7 bines. Esto es, la línea de código:

`TH1F *mass_four_lep      = new TH1F("mass_four_lep",    "Mass of four-lepton system; m_{4l} [GeV];Events / bin", 7, 110, 135);`

In [ ]:
%jsroot off

In [ ]:
#include<TFile.h>
#include<TH1.h>
#include<TMath.h>
#include<math.h>
#include<TF1.h>
#include<TLine.h>
#include<TStyle.h>
#include<TPoint.h>
#include<iostream>
#include<string>
#include<stdio.h>

#### Selección función de densidad de probabilidad (PDF, por sus siglas en inglés) a ajustar mediante el método de máxima verosimilitud (Likelihood, por su traducción del inglés) sobre los datos:

In [ ]:
const Int_t fit = 2; // Opciones: 1: Gauss. 2: Breit Wigner.

Función que guarda histogramas en formato .png, .pdf y .root en la caperta "Optimization/histograms/":

In [ ]:
void PrintCanvas(TCanvas* c1, string title) {
    string outFolder="histograms/";
    string tpng = outFolder+"/"+title+".png";
    c1->SaveAs(tpng.c_str());

    string tpdf = outFolder+"/"+title+".pdf";
    c1->SaveAs(tpdf.c_str());

    string troot = outFolder+"/"+title+".root";
    c1->SaveAs(troot.c_str());
    
    return;
}

#### Lectura y dibujado del histograma "mass_four_lep.root" obtenido en el cuaderno de Jupyter de "Plotting":

In [ ]:
TFile *f = new TFile("//../Plotting/histograms/mass_four_lep.root","READ");
//f->ls();
TH1F * h1 = (TH1F*)f->Get("c");

// Guardar histograma en formato .png y .root en la carpeta "Optimization/histograms/".
h1->SaveAs("histograms/mass_four_lep.png");
h1->SaveAs("histograms/mass_four_lep.root");

h1->Draw();

In [ ]:
TCanvas *c1 = new TCanvas("c1","c1",900, 600); // Creación de Canvas para la representación del ajuste.

Estilo de gráficos:

In [ ]:
gROOT->ForceStyle();

c1->cd();
//c1->SetGrid();

// Tamaño y margenes del pad
gStyle->SetPaperSize(20,26);
gStyle->SetPadTopMargin(0.05);
gStyle->SetPadRightMargin(0.05); 
gStyle->SetPadBottomMargin(0.16);
gStyle->SetPadLeftMargin(0.16);

// Distancia entre el eje y el título del eje
gStyle->SetTitleXOffset(1.4);
gStyle->SetTitleYOffset(1.4);

// Tamaño de título y etiquetas de los ejes X, Y y Z
gStyle->SetLabelSize(0.04,"XYZ");
gStyle->SetTitleSize(0.04,"XYZ");

// Formato de fuente
gStyle->SetTextFont(42);

//////////////////////////
// Legendas:
gStyle->SetOptTitle (0/1); // sin título
gStyle->SetOptStat(0); // No muestra estadística
gStyle->SetOptFit(1100); // Muestra chi^2/número de grados de libertar y p-value

gStyle->SetLegendBorderSize(0); // Tamaño de bordes de la leyenda
gStyle->SetStatFontSize(0.02); // tamaño de fuentes
gStyle->SetStatW(0.15); // ancho de la leyenda
gStyle->SetStatBorderSize(0); // Tamaño de bordes de la leyenda con información del ajuste
gStyle->SetStatX(.41); gStyle->SetStatY(.81); // Posición

gStyle->SetPalette(1); //opcional: cambia la paleta de colores en los plots.

Rango del histograma: entre 110 GeV y 135 GeV.

In [ ]:
Float_t minVal = 110.0; //Valor mínimo del intervalo.
Float_t maxVal = 135.0; //Valor máximo del intervalo.

#### Construcción de histograma de candidatos a bosón de Higgs. El tratamiento se dio al histograma mostrado en la celda anterior, al que se sustrajo el fondo simulado de MC (barras azul y morada) a los datos experimentales. Obteniéndose:

In [ ]:
// Representación de eventos candidatos a bosón de Higgs:
Int_t n = 7;
Int_t nbins = 7;
TH1D *data = new TH1D("data","data",n,minVal,maxVal); // Histograma de datos.

// Con 7 bins:
if (n == 7) {  
    data->SetBinContent(1, 1);
    data->SetBinContent(2, 0);
    data->SetBinContent(3, 1);
    data->SetBinContent(4, 8);
    data->SetBinContent(5, 4);
    data->SetBinContent(6, 0);
    data->SetBinContent(7, 2);
}

// Ajuste de etiquetas y estilo
data->SetName("data");
data->GetXaxis()->SetTitle("m_{4l} [GeV]");
c1->SetBottomMargin(0.12);
data->GetYaxis()->SetTitle("Events/5 GeV");
c1->SetLeftMargin(0.12);
data->SetAxisRange(0,11,"y");
data->SetLineColor(kBlack);
data->SetLineWidth(1);
data->SetMarkerStyle(21);
data->SetMarkerSize(1);
data->SetMarkerColor(kBlack);

Definición de PDF’s y parámetros a usar:

In [ ]:
if (fit == 1) {
    //PDF de la señal, más info: https://root.cern.ch/root/html524/TMath.html#TMath:Gaus
    TF1 *signalModel = new TF1("signalModel","[2]*TMath::Gaus(x,[0],[1],1)",minVal,maxVal);
    signalModel->SetParNames ("mean","sigma","constant"); //nombres de los parámetros
    signalModel->SetParameters(125.0,5.0,1.0); //valores de los parámetros
    //signalModel->FixParameter(0,123.8);
    signalModel->SetLineColor(kBlack);
    signalModel->SetLineWidth(2);
}
if (fit == 2) {
    //PDF de la señal, más info: https://root.cern.ch/root/html524/TMath.html#TMath:BreitWigner
    TF1 *signalModel = new TF1("signalModel","[2]*TMath::BreitWigner(x,[0],[1])",minVal,maxVal);
    signalModel->SetParNames("mass","width","constant"); //nombres de los parámetros
    signalModel->SetParameters(125.0,5.0,1.0); //valores de los parámetros
    //signalModel->FixParameter(0,123.8);
    signalModel->SetLineColor(kBlack);
    signalModel->SetLineWidth(2);
}

Configuración del ajuste de Likelihood y pintado de los resultados del ajuste:

In [ ]:
r = data->Fit("signalModel",  "LQS", "", minVal, maxVal); // L: likelihood, Q: quiet, reduce printing, S: save fit result
r->Print("v");
data->Draw("E");
//r->Print();

Cálculo del p-value, conocido $\chi^{2}$:

In [ ]:
cout << "p-value = " << TMath::Prob(signalModel->GetChisquare(),signalModel->GetNDF()) << endl;

Si el ajuste es con la PDF de Gauss, calcula la anchura a media altura (FWHM, por sus siglas en inglés) con su incertidumbre relativa.

In [ ]:
if (fit == 1) cout << "FWHM = " << 2.35*signalModel->GetParameter(1) << " +/- " << 2.35*signalModel->GetParError(1) << " GeV" << endl;

Representación de granja de intervalo de confianza $\pm 1\sigma$ (verde) y $\pm 2\sigma$ (gris):

In [ ]:
TGraphErrors *gr = new TGraphErrors(data);
Int_t nPoints = data->GetNbinsX();
for (Int_t i=0; i<nPoints; i++){
    gr->SetPoint(i, data->GetBinCenter(i+data->FindBin(110)), 0); 
}
(TVirtualFitter::GetFitter())->GetConfidenceIntervals(gr,0.68);
gr->SetName("gr");
gr->SetFillColor(3);

if (fit == 2) {
    TGraphErrors *gr2 = new TGraphErrors(data);
    for (Int_t i=0; i<nPoints; i++){
        gr2->SetPoint(i, data->GetBinCenter(i+data->FindBin(110)), 0); 
    }
    (TVirtualFitter::GetFitter())->GetConfidenceIntervals(gr2,0.95);
    gr2->SetName("gr2");
    gr2->SetFillColor(17);
    //gr2->SetFillStyle(3002);
    gr2->Draw("E3");
}

gr->Draw("E3");
data->Draw("E1P SAME");

In [ ]:
// Creación y ajuste de leyenda
auto legend = new TLegend(.63,.64,.90,.87);
legend->AddEntry("data","data","lep");
if (fit == 1) legend->AddEntry(signalModel,"Gaussian fit","L");
if (fit == 2) legend->AddEntry(signalModel,"Breit Wigner fit","L");
legend->AddEntry("gr","#pm 1 #sigma","f");
if (fit == 2) legend->AddEntry("gr2","#pm 2 #sigma","f");
legend->Draw("SAME");

// Pintado del histograma
c1->Draw();
PrintCanvas(c1, "fit"); // Guardar fit en formato PNG, PDF y ROOT.

Creación de "mapa de calor" que representa el ancho de la señal como función de la masa invariante de los eventos reales candidatos a bosón de Higgs, para distintos valores de $− 𝑙𝑛(\mathcal{𝐿})$.

In [ ]:
Float_t stepMass = 0.5;
Float_t stepWidth = 0.25;
binMass = int((maxVal-minVal)/stepMass);
binWidth = int(10./stepWidth);
L_Plot = new TH2D("L_Plot","L_Plot",binMass,minVal-stepMass/2.0,maxVal-stepMass/2.0,binWidth,0.5,10.5);

cout << "binMass: " << binMass << endl;
cout << "binWidth: " << binWidth << endl;

Ajuste de fijando tanto la masa como la anchura, de forma que nos permitan obtener el $− 𝑙𝑛(\mathcal{𝐿})$ en función de dichos parámetros. Sin fijarlos, se obtendría siempre el valor que más se ajuste a los datos, esto es: el mínimo.

In [ ]:
L_Plot->Clear();
for (Float_t masses = minVal; masses < maxVal; masses+=stepMass){
    //cout << "Filling: externo mass: "<< masses << ". bin:" << L_Plot->GetXaxis()->FindBin(masses) << endl;
    for (Float_t widths = 0.25; widths < 10.5; widths+=stepWidth){
        //cout << "Filling: interno mass and bin: " << widths << L_Plot->GetYaxis()->FindBin(widths) << endl;
        
        for (Int_t i=0; i<3; i++) signalModel->ReleaseParameter(i);
        signalModel->FixParameter(0,masses);
        signalModel->FixParameter(1,widths);
        r = data->Fit("signalModel","QLS");
        L_Plot->SetBinContent(L_Plot->FindBin(masses,widths),r->MinFcnValue());
    }
}

In [ ]:
// Pintado y ajustes de "mapa de calor"
L_Plot->SetAxisRange(0.5,10.4,"y");
L_Plot->SetAxisRange(0,30,"z");
L_Plot->SetAxisRange(110,134.7,"x");
c1->SetRightMargin(0.15);
L_Plot->GetXaxis()->SetTitle("m_{4l} [GeV]");
L_Plot->GetYaxis()->SetTitle("Width of the signal [GeV]");
c1->SetRightMargin(0.15);
L_Plot->GetZaxis()->SetTitle("-ln(L)");
L_Plot->Draw("colz");
gStyle->SetOptStat(000000);
c1->Draw();
PrintCanvas(c1, "2D_profile_likelihood"); // Guardar 2D_profile_likelihood en formato .png, .pdf y .root.

#### Representaciones extra:
Proyección y perfil en el eje X e Y

In [ ]:
auto c2 = new TCanvas("Canvas2","Canvas2",900,600);
c2->Divide(2,2);
c2->cd(1); L_Plot->ProjectionX()->Draw("E");
c2->cd(2); L_Plot->ProjectionY()->Draw("E");
c2->cd(3); L_Plot->ProfileX()->Draw("E");
c2->cd(4); L_Plot->ProfileY()->Draw("E");
c2->Draw();

Proyección: https://root.cern.ch/doc/master/classTH2.html#a974ece9e7d260f92df00a39dba14e5b0

Perfil: https://root.cern.ch/doc/master/classTH2.html#a19eb4ae4f4d399b0f4ad820d838de75e

In [ ]:
// Mapa de calor en 3D

c1->cd();

// remove borders..
L_Plot->SetAxisRange(0.5,10,"y");

L_Plot->Draw("surf2");
c1->Draw();
PrintCanvas(c1, "3D_profile_likelihood_1"); // Guardar 3D_profile_likelihood_1 en formato .png, .pdf y .root.

In [ ]:
// // Mapa de calor en 3D

L_Plot->Draw("surf3");
c1->Draw();
PrintCanvas(c1, "3D_profile_likelihood_2"); // Guardar 3D_profile_likelihood_2 en formato .png, .pdf y .root.